## Convert two segmentation results to RAGs, fuse them and perform inverse operation to get fused segmentation result

In [ ]:
from skimage.future.graph import RAG
from skimage import data, io, segmentation, color,filters
from matplotlib import pyplot as plt, colors
from skimage import morphology, measure, io
from skimage.measure import label, regionprops
import tifffile as tiff
import numpy as np
import networkx as nx
import pandas as pd
from skimage.future import graph

In [ ]:
i= io.imread('path to original raw image/*.tif')
s1= io.imread('path to segmentation result by P1 in .tif')
s2= io.imread('path to segmentation result by P2 in .tif')


In [ ]:
def rag(image, labels):
   #initialize the RAG
   graph = RAG(labels, connectivity=2)

   for n in graph:
       graph.nodes[n].update({'labels': [n],'pixel count': 0,
                             'total color': np.array([0, 0, 0],
                             dtype=np.double)})
   #give them values
   for index in np.ndindex(labels.shape):
       current = labels[index]
       graph.nodes[current]['pixel count'] += 1
       graph.nodes[current]['total color'] += image[index]

   #calculate your own weights here
   for x, y, d in graph.edges(data=True):
       my_weight = 1  ## setting to 1 now
       d['weight'] = my_weight

   return graph

def node2label(labels, rag):
    label_map = np.arange(labels.max() + 1)
    for ix, (n, d) in enumerate(rag.nodes(data=True)):
        for label in d['labels']:
            label_map[label] = ix

    return label_map[labels]

In [ ]:
### Forward
rag1= rag(i,s1)
rag2= rag(i,s2)

F = nx.compose(rag2,rag1) 

labels_fus= node2label(s1+s2, F)
labels_fus=labels_fus.astype(np.uint16)

tiff.imsave('ragfused_resumt.tif', labels_fus)
